### Run Whole Evaluation

In [2]:
from sentence_transformers import SentenceTransformer, util
import json
from tqdm import tqdm
import numpy as np
import evaluate
from nltk.tokenize import word_tokenize
import random
import nltk
from scipy.spatial.distance import jensenshannon
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import jensenshannon

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


/home/wenkail/miniconda3/envs/lp/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/wenkail/miniconda3/envs/lp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# Read Ground True File
# with open("testset/alpaca_big_five_dataset_test.json", 'r') as f:
#     ground_true = json.load(f)
# "testset/aplaca_big_five_dataset_train_first_1000_5_tokens.json"

ground_true_file_path = "testset/alpaca_big_five_dataset_1000_test_5_tokens_subtestset.json"
with open(ground_true_file_path, 'r') as f:
    ground_true = json.load(f)
    # Subsample here
    # num_samples = 1000
    # seed = 42
    # random.seed(seed)
    # ground_true = random.sample(ground_true, num_samples)

# Read Prediction File
prediction_file_path = "testset/llama_3b_persona_inferene_1000_token_5_epoch_subtestset.json"
with open(prediction_file_path, 'r') as f:
    prediction = json.load(f)

# # Read Prediction File
# with open("testset/llama_3b_persona_inferene_result_epoch_40.json") as f:
#     prediction_40 = json.load(f)

def compute_token_overlap_ratio(text1, text2):
    tokens1 = set(word_tokenize(text1.lower()))
    tokens2 = set(word_tokenize(text2.lower()))
    overlap = len(tokens1.intersection(tokens2))
    ratio = overlap / max(len(tokens1), len(tokens2))
    return ratio

def compute_jsd(sentence1, sentence2):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    sentences = [sentence1, sentence2]
    embeddings = model.encode(sentences)
    def normalize_vector(vector):
        vector = np.abs(vector)
        return vector / np.sum(vector)
    norm_embeddings = np.array([normalize_vector(embedding) for embedding in embeddings])
    jsd = jensenshannon(norm_embeddings[0], norm_embeddings[1], base=2)
    similarity = 1 - jsd
    return similarity

In [5]:
print(ground_true[0]['input'] + ' ' + ground_true[0]['output'])
print(prediction[0]['input'] + " " + prediction[0]['prediction'])

there is a midget in a wheelchair parked in the hallway outside my office.i suppose that he's there waiting for the meeting in the conference room to let out, or something.
there is a midget in my room


In [6]:
#Compute embedding to gain similarity
similarity_list = []
for i in tqdm(range(len(prediction)), desc = "Proccessing Cosine Similarity: "):
    embedding_1= model.encode(ground_true[i]['input'] + " " + ground_true[i]['output'], convert_to_tensor=True)
    embedding_2 = model.encode(prediction[i]['input'] + " " + prediction[i]['prediction'], convert_to_tensor=True)
    similarity_list.append(util.pytorch_cos_sim(embedding_1, embedding_2).cpu().detach().numpy())
print(f"Average Cosine Sentence Similarity: {np.mean(similarity_list)}")

overlap_ratios = []
for i in tqdm(range(len(prediction)), desc = "Proccessing Token Overlapping: "):
    true_text = ground_true[i]['input'] + " " + ground_true[i]['output']
    pred_text = prediction[i]['input'] + " " + prediction[i]['prediction']
    
    overlap_ratio = compute_token_overlap_ratio(true_text, pred_text)
    overlap_ratios.append(overlap_ratio)
average_overlap_ratio = np.mean(overlap_ratios)
print(f"Average Token Overlapping Ratio: {average_overlap_ratio}")

jsd_list = []
for i in tqdm(range(len(prediction)), desc = "Proccessing JSD Similarity: "):
    true_text = ground_true[i]['input'] + " " + ground_true[i]['output']
    pred_text = prediction[i]['input'] + " " + prediction[i]['prediction']
    
    jsd_score = compute_jsd(true_text, pred_text)
    jsd_list.append(jsd_score)

average_jsd_score = np.mean(jsd_list)
print(f"Average JSD Sentence Similarity: {average_jsd_score}")

Proccessing Cosine Similarity: 100%|██████████| 998/998 [00:08<00:00, 116.84it/s]


Average Cosine Sentence Similarity: 0.520480751991272


Proccessing Token Overlapping: 100%|██████████| 998/998 [00:00<00:00, 2783.47it/s]


Average Token Overlapping Ratio: 0.36071023577639433


Proccessing JSD Similarity: 100%|██████████| 998/998 [08:52<00:00,  1.88it/s]

Average JSD Sentence Similarity: 0.6142571672783119


In [7]:
cosine_similarity = np.mean(similarity_list)
print("Average Cosine Sentence Similarity: {:.4f}".format(cosine_similarity))
print("Average Token Overlapping Ratio: {:.4f}".format(average_overlap_ratio))
print("Average JSD Sentence Similarity: {:.4f}".format(average_jsd_score))

Average Cosine Sentence Similarity: 0.5205
Average Token Overlapping Ratio: 0.3607
Average JSD Sentence Similarity: 0.6143


### Test Evaluation

In [19]:
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import jensenshannon
import numpy as np

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

sentences = ["This is the first sentence.", "This is the third sentence."]

embeddings = model.encode(sentences)

def normalize_vector(vector):
    vector = np.abs(vector)
    return vector / np.sum(vector)

norm_embeddings = np.array([normalize_vector(embedding) for embedding in embeddings])

jsd = jensenshannon(norm_embeddings[0], norm_embeddings[1], base=2)


similarity = 1 - jsd
return similarity


Jensen-Shannon Divergence: 0.36065374499994163
Similarity (1 - JSD): 0.6393462550000584
